# BERT for fake news detection


Make sure that you are using GPU

In [ ]:
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

You can see the type of gpu available for you:

In [ ]:
!nvidia-smi

Thu Mar 24 23:14:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Load the data

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
root = 'drive/My Drive/Colab Notebooks/'
reda_root = 'drive/My Drive/TFG/'
paht = 'drive/My Drive/Colab Notebooks/fakkedit/'

Mounted at /content/drive


In [ ]:
MULTIMODAL_ONLY = True # if False, we will load all the
nameFile = 'multimodal_'
if not MULTIMODAL_ONLY:
    nameFile ='all_'


In [ ]:
import pandas as pd
train = pd.read_csv(reda_root+"data/fakeddit/" + nameFile+ "train.tsv", sep='\t')
val  = pd.read_csv(reda_root+"data/fakeddit/" + nameFile+ "validate.tsv", sep='\t')
test  = pd.read_csv(reda_root+"data/fakeddit/" + nameFile+ "test_public.tsv", sep='\t')

print("Dataset: ", nameFile)
print('size training: ', len(train))
print('size validation: ', len(val))
print('size test: ', len(test))


Dataset:  multimodal_
size training:  564000
size validation:  59342
size test:  59319


To make faster experimentation, we reduce the size of the datasets (of course, the results will be worse)

In [ ]:
import numpy as np
reduction = 10 #100, 10 or 1

print("reduction applied: ", reduction)


print(f"Training patterns before reduction: {len(train)}")
train = train.sample(int(len(train)/reduction), random_state=12345)
print(f"Training patterns after reduction:  {len(train)}")

print(f"Validation patterns before reduction: {len(val)}")
val = val.sample(int(len(val)/reduction), random_state=12345)
print(f"Validation patterns after reduction:  {len(val)}")

print(f"Test patterns before reduction: {len(test)}")
test = test.sample(int(len(test)/reduction), random_state=12345)
print(f"Test patterns after reduction:  {len(test)}")

reduction applied:  10
Training patterns before reduction: 564000
Training patterns after reduction:  56400
Validation patterns before reduction: 59342
Validation patterns after reduction:  5934
Test patterns before reduction: 59319
Test patterns after reduction:  5931


In [ ]:
train.head()

,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
224716,T3nd0o,this basket made out of old candy wrappers fea...,1.566649e+09,i.redd.it,True,cusqgc,https://preview.redd.it/vz64u4m03ei31.jpg?widt...,NaN,2.0,20,mildlyinteresting,This basket made out of old candy wrappers (fe...,0.80,1,0,0
525216,TindalosKeeper,ugh shitty connection,1.560360e+09,NaN,True,eqwq1xm,https://i.imgur.com/jhHIGoj.jpg,bzptls,NaN,8,psbattle_artwork,"Ugh, shitty connection!",NaN,0,2,4
104777,ZAPP3Rx,evil racoon,1.508945e+09,i.redd.it,True,78od95,https://preview.redd.it/as247oqduztz.jpg?width...,NaN,0.0,4,pareidolia,Evil racoon,0.70,0,2,2
467061,Count_Mirchaud,german tank ace michael wittmans last stand ag...,1.481311e+09,imgur.com,True,5hfnof,https://external-preview.redd.it/F4ThHaQXXlqf3...,NaN,0.0,9,fakehistoryporn,German Tank Ace Michael Wittman's last stand a...,0.92,0,2,2
350361,GuysnDolls,the dread of lancaster bay,1.552235e+09,NaN,True,ei7pl4z,https://i.imgur.com/RgHJ9D3.jpg,azb1dq,NaN,11,psbattle_artwork,The Dread of Lancaster Bay,NaN,0,2,4


We only keep the clean title and the columns for classification:

In [ ]:
train_data = train[["clean_title", "2_way_label", "3_way_label", "6_way_label"]]
val_data=val[["clean_title", "2_way_label", "3_way_label", "6_way_label"]]
test_data=test[["clean_title", "2_way_label", "3_way_label", "6_way_label"]]
# remove records with Nan values
train_data = train_data.dropna()
val_data = val_data.dropna()
test_data = test_data.dropna()


In [ ]:
train_data.head()

,clean_title,2_way_label,3_way_label,6_way_label
224716,this basket made out of old candy wrappers fea...,1,0,0
525216,ugh shitty connection,0,2,4
104777,evil racoon,0,2,2
467061,german tank ace michael wittmans last stand ag...,0,2,2
350361,the dread of lancaster bay,0,2,4


We save the texts:

In [ ]:
X_train = train_data["clean_title"].values.tolist()
X_val = val_data["clean_title"].values.tolist()
X_test = test_data["clean_title"].values.tolist()
print(type(X_train))

<class 'list'>


We show some examples:

In [ ]:
print(X_train[0])
print(X_val[0])
print(X_test[0])


this basket made out of old candy wrappers feat some hot garlic bread
catana contender for mortal kombat
cutout


We decide what type of classication we will address: 
- binary (2_way_label)
- tres classes (3_way_label)
- 6 classes (6_way_label)

In [ ]:
TYPE_CLASSIFICATION = 6 # 2, 3 OR 6

In [ ]:
y_train = train_data[str(TYPE_CLASSIFICATION) + '_way_label'].values.tolist()
y_val = val_data[str(TYPE_CLASSIFICATION) + '_way_label'].values.tolist()
y_test = test_data[str(TYPE_CLASSIFICATION) + '_way_label'].values.tolist()


We do not need to encoder the labels because they are already provided as numbers. 

That is, in the 6_way_label, the labels are 0, 1, 2, 3, 4, 5. 
- 0: true
- 1: Satire/Parody:
- 2: Misleading Content
- 3: Imposter Content
- 4: False Connection
- 5: Manipulated Content

In the 3_way_label, the labels are 0, 1, 2:
- 0: true
- 1: the sample is fake and contains text that is true (i.e. direct quotes from propaganda posters)
- 2: False

In the 2_way_label, the labels are 0, 1:
- 0: true
- 1: False


In [ ]:
labels_dict = {}
if TYPE_CLASSIFICATION == 2:
    labels_dict[0] = "True"
    labels_dict[1] = "False"
elif TYPE_CLASSIFICATION == 3:
    labels_dict[0] = "True"
    labels_dict[1] = "Fake contains True"
    labels_dict[2] = "False"
elif TYPE_CLASSIFICATION == 6:
    labels_dict[0] = "True"
    labels_dict[1] = "Satire/Parody"
    labels_dict[2] = "Misleading Content"
    labels_dict[3] = "Imposter Content"
    labels_dict[4] = "False Connection"
    labels_dict[5] = "Manipulated Content"

print(labels_dict)

{0: 'True', 1: 'Satire/Parody', 2: 'Misleading Content', 3: 'Imposter Content', 4: 'False Connection', 5: 'Manipulated Content'}


We show some examples and their labels:

In [ ]:
print(X_train[0], y_train[0], labels_dict[y_train[0]])
print(X_val[0], y_val[0], labels_dict[y_val[0]])
print(X_test[0], y_test[0], labels_dict[y_test[0]])




this basket made out of old candy wrappers feat some hot garlic bread 0 True
catana contender for mortal kombat 4 False Connection
cutout 4 False Connection


## Data Visualization

Here we should provide more detail about the dataset (class distribution, average lenght of the texts, etc)

In [ ]:
# we will do it after

## Transformers

First, we install the libraries:

In [ ]:
!pip3 install transformers numpy torch sklearn


     |████████████████████████████████| 3.8 MB 30.2 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 6.5 MB 61.2 MB/s 
     |████████████████████████████████| 895 kB 54.9 MB/s 
     |████████████████████████████████| 596 kB 67.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


To make our experiments reproducible, we set a random seed:


In [ ]:
import torch
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

Import the necessary modules:

In [ ]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments



we'll be using the BERT model. More specifically, we'll be using bert-base-uncased pre-trained weights from the library.

In [ ]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# max sequence length for each document/sentence sample
max_length = 512

Download the tokenizer provided by this model:

In [ ]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Now let's use our tokenizer to encode our corpus. This tokenizer translate the tokens to ids. We set truncation to True so that we eliminate tokens that go above max_length, we also set padding to True to pad documents that are less than max_length with empty tokens.

In [ ]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)


The below code wraps our tokenized text data into a torch Dataset. Since we gonna use Trainer from Transformers library, it expects our dataset as a torch.utils.data.Dataset, so we made a simple class that implements the len() method that returns the number of samples, and getitem() method to return a data sample at a specific index.

In [ ]:
class TorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_torch_dataset = TorchDataset(train_encodings, y_train)
val_torch_dataset = TorchDataset(val_encodings, y_val)
test_torch_dataset = TorchDataset(test_encodings, y_test)

Below is an example usage of this tokenizer on the first headline. Here MAX_LEN specifies the length of each tokenized review. If a tokenized review is shorter than MAX_LEN, then it is padded with zeros. If a tokenized review is greater than MAX_LEN, then it is truncated (this is why we have truncation=True and padding=True).

In [ ]:
test = X_train[0]
inputs = tokenizer(test, max_length=max_length, truncation=True, padding=True)

print(f'review: \'{test}\'')
print(f'input ids: {inputs["input_ids"]}')
print(f'attention mask: {inputs["attention_mask"]}')

review: 'this basket made out of old candy wrappers feat some hot garlic bread'
input ids: [101, 2023, 10810, 2081, 2041, 1997, 2214, 9485, 10236, 7347, 8658, 2070, 2980, 20548, 7852, 102]
attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(len(train_encodings.encodings), len(y_train))

56400 56400


## Training the model

Now that we have our data prepared, let's download and load our BERT model and its pre-trained weights. We're using BertForSequenceClassification class from Transformers library, we set num_labels to the length of our available labels, in this case, 2.

We also cast our model to our CUDA GPU. If you're on CPU (not suggested), then just delete to() method.

In [ ]:
# load the model and pass to CUDA
# cuda not working with TFDistilBertForSequenceClassification ??????
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = TYPE_CLASSIFICATION, ignore_mismatched_sizes=True).to('cuda')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Before we start fine-tuning our model, let's make a simple function to compute the metrics we want. In this case, accuracy.
You're free to include any metric you want, I've included accuracy, but you can add precision, recall, etc.

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,   # 3           # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer = Trainer(
    model = model,                         # the instantiated Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_torch_dataset,         # training dataset
    eval_dataset = val_torch_dataset,          # evaluation dataset
    compute_metrics = compute_metrics,     # the callback that computes metrics of interest
)

# train the model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 56400
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7050


Step,Training Loss,Validation Loss,Accuracy
400,1.384300,1.029990,0.633805
800,0.975000,0.954273,0.671722
1200,0.898200,0.868886,0.688911
1600,0.857300,0.802182,0.714358
2000,0.796100,0.813695,0.719077
2400,0.780500,0.761200,0.735423
2800,0.809900,0.758068,0.732221
3200,0.753500,0.756903,0.736266
3600,0.775900,0.730399,0.737782
4000,0.757800,0.707462,0.747556


***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-800
Configuration saved in ./results/checkpoint-800/config.json
Model weights saved in ./results/checkpoint-800/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1200
Configuration saved in ./results/checkpoint-1200/config.json
Model weights saved in ./results/checkpoint-1200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1600
Configuration saved in ./results/checkpoint-1600/config.json
Model weights saved in ./results/checkpoint-1600/pytorch_model.bi

TrainOutput(global_step=7050, training_loss=0.8010544905425809, metrics={'train_runtime': 1309.2037, 'train_samples_per_second': 43.08, 'train_steps_per_second': 5.385, 'total_flos': 2889444254716800.0, 'train_loss': 0.8010544905425809, 'epoch': 1.0})

In [ ]:
# evaluate the current model after training
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20


{'epoch': 1.0,
 'eval_accuracy': 0.7655881361644759,
 'eval_loss': 0.6646504402160645,
 'eval_runtime': 10.783,
 'eval_samples_per_second': 550.309,
 'eval_steps_per_second': 27.543}

Save the model

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "distilbert-base-uncased-finetuned-sst-2-english"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in distilbert-base-uncased-finetuned-sst-2-english/config.json
Model weights saved in distilbert-base-uncased-finetuned-sst-2-english/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-sst-2-english/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst-2-english/special_tokens_map.json


('distilbert-base-uncased-finetuned-sst-2-english/tokenizer_config.json',
 'distilbert-base-uncased-finetuned-sst-2-english/special_tokens_map.json',
 'distilbert-base-uncased-finetuned-sst-2-english/vocab.txt',
 'distilbert-base-uncased-finetuned-sst-2-english/added_tokens.json',
 'distilbert-base-uncased-finetuned-sst-2-english/tokenizer.json')

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    # return probs.argmax() is a tensor. We have to return its item
    return probs.argmax().item()


In [ ]:
y_pred=[]
for text in X_test:
    y_pred.append(get_prediction(text))


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

print(labels_dict)
print(classification_report(y_test, y_pred))

{0: 'True', 1: 'Satire/Parody', 2: 'Misleading Content', 3: 'Imposter Content', 4: 'False Connection', 5: 'Manipulated Content'}
              precision    recall  f1-score   support

           0       0.79      0.87      0.83      2351
           1       0.59      0.54      0.56       376
           2       0.73      0.54      0.62      1154
           3       0.39      0.22      0.28       139
           4       0.77      0.85      0.81      1675
           5       0.73      0.65      0.69       236

    accuracy                           0.76      5931
   macro avg       0.67      0.61      0.63      5931
weighted avg       0.75      0.76      0.75      5931



In [ ]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labels_dict)
disp.plot()
plt.show()

We show some predictions for new texts. The second prediction is wrong (the news is fake). 

In [ ]:
text = "Donald Trump sent his own plane to transport 200 stranded marines."

pred=get_prediction(text)
print('class:', labels_dict[pred])

text = "FBI director received millions from Clinton Foundation, his brother’s law firm does Clinton’s taxes"

pred=get_prediction(text)
print('class:', labels_dict[pred])

class: Misleading Content
class: True
